In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
os.environ['WDM_LOG_LEVEL'] = '0'

# 
s = Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=s, chrome_options=chrome_options)

driver.set_page_load_timeout(100)

url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/chikun.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/zika.def"
driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))

# Linha
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[1]/select/option[1]').click()
# Coluna
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[8]').click()
# Periodo
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[2]').click()
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()
# Mostra
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()


WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[1]/a')))

html = driver.page_source

driver.quit()

/tmp/ipykernel_35438/369096376.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=chrome_options)


In [10]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [11]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

In [12]:
df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [14]:
aux = list(df.iloc[:, 0])
for i in range (len(aux)):
    aux[i] = aux[i].split(" ")
    while (len(aux[i]) > 2):
        aux[i][1] = aux[i][1] + " " + aux[i][2]
        aux[i].pop(2)
aux = pd.DataFrame(aux, columns=["Codigo", "Município"])
df.iloc[:, 0] = aux["Município"]

In [16]:
colunas = [df["<1 Ano"], df["1-4"], df["5-9"], df["10-14"], df["15-19"], df["20-34"], df["35-49"], df["50-64"], df["65-79"], df["80 e+"]]

for i in range (len(colunas)):
    for j in range (len(colunas[i])):
        try:
            colunas[i][j] = int(colunas[i][j])
        except:
            colunas[i][j] = 0

In [17]:
df.to_csv("../scr/fx_etaria_municipio.csv", encoding='utf-8')

In [18]:
df

,Município,<1 Ano,1-4,5-9,10-14,15-19,20-34,35-49,50-64,65-79,80 e+
0,Abaíra,0,0,0,0,0,0,0,1,0,0
1,Abaré,0,0,0,0,1,1,0,0,0,0
2,Acajutiba,0,0,0,0,0,3,0,0,0,0
3,Adustina,0,0,0,0,1,4,2,2,0,0
4,Água Fria,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
345,Vitória da Conquista,29,25,23,25,34,277,221,95,30,8
346,Wagner,0,1,0,0,0,2,1,3,0,0
347,Wanderley,0,0,0,0,0,0,0,1,0,0
348,Wenceslau Guimarães,0,0,0,0,0,0,1,0,1,0
